Camel 3 Function 
=====================


In [1]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

Environment detected: local
Setting up local development environment...


  Activating project at `~/globtim/environments/local`


Loading CairoMakie...
CairoMakie activated for high-quality plots
GLMakie available for interactive plots
Loading Globtim from main project...
Globtim loaded successfully!
Ready for local development!
Available: Full plotting, interactive development tools
Switch plotting: GLMakie.activate!() for interactive plots


We define the center and range of the square domain of approximation. 

In [2]:
# Constants and Parameters
const n, a, b = 2, 5, 1
const scale_factor = a / b
f = camel # Objective function
d = 6 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

Number of samples: 40000


test_input(2, [0.0, 0.0], 200, (0.1, 0.5), 0.002, (0.0, 0.0), 5.0, 1.0, 6, Globtim.camel)

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision. 

In [3]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

current L2-norm: 5.4524172392659695e-12
current L2-norm: 4.6461539462782044e-12


Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [4]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

Tracking 25 paths...   8%|██▍                           |  ETA: 0:00:36



Tracking 25 paths... 100%|██████████████████████████████| Time: 0:00:03
                   # paths tracked: 25
   # non-singular solutions (real): 25 (17)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 25 (17)
Tracking 25 paths...   8%|██▍                           |  ETA: 0:00:09



Tracking 25 paths... 100%|██████████████████████████████| Time: 0:00:00
                   # paths tracked: 25
   # non-singular solutions (real): 25 (17)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 25 (17)


In [5]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.00125);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.00125);

Processing point 1 of 15
Optimization has converged within bounds: ✓
Processing point 2 of 15
Optimization has converged within bounds: ✓
Processing point 3 of 15
Optimization has converged within bounds: ✓
Processing point 4 of 15
Optimization has converged within bounds: ✓
Processing point 5 of 15
Optimization has converged within bounds: ✓
Processing point 6 of 15
Optimization has converged within bounds: ✓
Processing point 7 of 15
Optimization has converged within bounds: ✓
Processing point 8 of 15
Optimization has converged within bounds: ✓
Processing point 9 of 15
Optimization has converged within bounds: ✓
Processing point 10 of 15
Optimization has converged within bounds: ✓
Processing point 11 of 15
Optimization has converged within bounds: ✓
Processing point 12 of 15
Optimization has converged within bounds: ✓
Processing point 13 of 15
Optimization has converged within bounds: ✓
Processing point 14 of 15
Optimization has converged within bounds: ✓
Processing point 15 of 15
Opt

In [6]:
fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb, chebyshev_levels=true)

In [7]:
fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true)
# Commented out save command
# save("camel_lege_levelset.pdf", fig_2)